In [35]:
import functorch as ft
import torch
from torch.utils.benchmark import Timer

import sys; sys.path.append('../src/')
from model.cmpnts import MLP

# JVP
## batched

In [78]:
z = torch.randn(32, 100).cuda()
v = torch.randn(32, 100).cuda()
mlp = MLP(100, 256*256, [32]*4).cuda()

def fn(z):
    return mlp(z)

In [79]:
without_vmap = Timer(stmt="ft.jvp(fn, (z,), (v,))", globals=globals())
without_vmap.timeit(500)

ft.jvp(fn, (z,), (v,))
  1.60 ms
  1 measurement, 500 runs , 1 thread

## vmap

In [80]:
def fn_single(z):
    return mlp(z.unsqueeze(0))[0]

def jvp_single(z, v):
    return ft.jvp(fn_single, (z,), (v,))

In [81]:
with_vmap = Timer(stmt="ft.vmap(jvp_single)(z, v)", globals=globals())
with_vmap.timeit(500)

ft.vmap(jvp_single)(z, v)
  2.02 ms
  1 measurement, 500 runs , 1 thread

In [82]:
torch.allclose(ft.jvp(fn, (z,), (v,))[1], ft.vmap(jvp_single)(z, v)[1])

True

# VJP
## Batched

In [48]:
x = torch.randn(128, 256*256)
u = torch.randn(128, 10)
mlp1 = MLP(256*256, 10, [32]*4)

def gn(x):
    return mlp1(x)

In [55]:
without_vmap = Timer(stmt="ft.vjp(gn, x)[1](u)", globals=globals())
without_vmap.timeit(500)

ft.vjp(gn, x)[1](u)
  16.46 ms
  1 measurement, 500 runs , 1 thread

In [190]:
torch.svd(torch.randn(10, 20), compute_uv=False)

torch.return_types.svd(
U=tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
S=tensor([6.6415, 5.8416, 5.2784, 4.1654, 3.7680, 3.5360, 3.1329, 2.9329, 2.2247,
        1.4396]),
V=tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0